## Жолковский Евгений

In [154]:
import pandas as pd
import numpy as np
import json
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from alphabet_detector import AlphabetDetector
from sklearn.naive_bayes import MultinomialNB
from mlxtend.preprocessing import TransactionEncoder
import xgboost

https://www.kaggle.com/c/sentence-boundary-detection-itmo-2018

In [4]:
train_data = []
with io.open('train_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        train_data.append(d)

In [5]:
test_data = []
with io.open('test_data.json','r',encoding='utf8') as f:
    for line in f.readlines():
        d = json.loads(line)
        test_data.append(d)

In [6]:
pos_ends = ['!', '?', '…', '.', "»", '"']

def convert_train(train_data):
    train1_data = []
    for tr in train_data:
        Marks = []
        D = {}
        positions = []
        par = ''
        ## generate true labels
        for s in tr['Sentences']:

            par += s + ' '
            pos = len(par)-2 
            D1 = {}  
            positions.append(len(par)-2)
            D1['Pos'] =  len(par)-2
            D1['Mark'] = par[pos]
            D1['End'] = True
            Marks.append(D1)

        

        # generate false labels
        for j, sym in enumerate(par):
            if (sym in pos_ends) and (not j in positions):
                D1 = {}  
                D1['Pos'] =  j
                D1['Mark'] = sym
                D1['End'] = False
                Marks.append(D1)
                
        D['Marks'] = Marks
        D['Paragraph'] = par[:-1]
        
        train1_data.append(D)
    return train1_data

In [219]:
def encode(st):
    ad = AlphabetDetector()
    enc = []
    for j, c in enumerate(st):

        if c.isalpha():     
            enc.append('a' + str(j))
            
            if ad.is_cyrillic(c):
                enc.append('c' + str(j))

            if c.isupper():
                enc.append('A' + str(j))


        elif c.isnumeric():
            enc.append('n' + str(j) )
            
        else:
            enc.append(c + str(j))
            enc.append('n' + str(j))


    return enc

In [220]:
train_data_conv = convert_train(train_data)

In [221]:
train, test = train_test_split(train_data_conv, test_size=0.33, random_state=41)

In [222]:
k = 5

In [223]:
X_train = []
y_train = []
for tr in train:
    par = tr['Paragraph']
    for cand in tr['Marks']:
        y_train.append(int(cand['End']))
        code = encode(par[cand['Pos']-k: cand['Pos']+k])
        X_train.append(code)
y_train = np.array(y_train)

In [224]:
X_test = []
y_test = []
y_test_mask = []
for tr in test:
    par = tr['Paragraph']
    for cand in tr['Marks']:
        code = encode(par[cand['Pos']-k: cand['Pos']+k])
        X_test.append(code)
        
        if cand['Pos'] == len(par)-1:
            y_test_mask.append(1)
        else:
            y_test_mask.append(0)
      
        y_test.append(int(cand['End'])) 
y_test = np.array(y_test)

In [225]:
X_subm = []
y_subm_mask = []
for tr in test_data:
    par = tr['Paragraph']
    for cand in tr['Marks']:
        code = encode(par[cand['Pos']-k: cand['Pos']+k])
        
        if cand['Pos'] == len(par)-1:
            y_subm_mask.append(1)
        else:
            y_subm_mask.append(0)
        
        X_subm.append(code) 

In [226]:
OH = TransactionEncoder()
OH.fit(X_train + X_subm + X_test) # это никак не влияет, но позволяет нормально закодировать

x_train = OH.transform(X_train).astype(int)
x_test = OH.transform(X_test).astype(int)
x_subm = OH.transform(X_subm).astype(int)



## logistic regression

In [227]:
clf_lr = LogisticRegression(C=10000, penalty='l1')

In [228]:
clf_lr.fit(x_train, y_train)

y_pred = clf_lr.predict(x_test)
print ((y_pred == y_test).mean())

for j, y_ in enumerate(y_test_mask):
    y_pred[j] |= y_
    
print ((y_pred == y_test).mean())

0.9865691489361702
0.9866023936170213


## xgboost

In [229]:
clf_xgb = xgboost.XGBClassifier(n_estimators=1500, max_depth=6, learning_rate=0.01, n_jobs=-1, silent=False)

In [230]:
clf_xgb.fit(x_train, y_train)

y_pred = clf_xgb.predict(x_test)
print ((y_pred == y_test).mean())

for j, y_ in enumerate(y_test_mask):
    y_pred[j] |= y_
    
print ((y_pred == y_test).mean())

0.9902593085106383
0.9902593085106383


/home/e.zholkovskiy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## submission

In [232]:
y_subm = clf_xgb.predict(x_subm)

for j, y_ in enumerate(y_subm_mask):
    y_subm[j] |= y_

/home/e.zholkovskiy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [233]:
df = pd.DataFrame(y_subm, columns=['Mark'], index=range(1,26477))
df.index.name = 'Id'
df.to_csv("sampleSubmission.csv")